In [10]:
import numpy as np
import pandas as pd
from astroquery.jplhorizons import Horizons
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import time
from astropy.coordinates import GeocentricMeanEcliptic
from astropy.coordinates import HeliocentricMeanEcliptic
from astropy.coordinates import HeliocentricTrueEcliptic
from astropy.coordinates import GeocentricTrueEcliptic
from astropy.coordinates import BarycentricTrueEcliptic
from astropy.coordinates import BarycentricMeanEcliptic
from astropy import coordinates 
import astropy

'''
    NAME:
         convert_to_primary_centric
         
    PURPOSE:
         This function takes a parameter Dataframe in RA/DEC, and converts it to Latitude 
         and Longitude, while also converting the dates to Primary-Centric Julian dates
         
    CALLING SEQUENCE:
         convert_to_primary_centric(paramsDF, objectName)
   
    INPUTS
          paramsDF - A dataframe of the observed positional data of the KBO in question
          objectName - The name of the object being observed (needed for the Horizons function)
   
    OUTPUTS:
          None. Just makes plots currently.
'''
def convert_to_primary_centric(paramsDF,objectName):
     #Current column names are just descriptive, not representative of final product column names
    updatedDF = pd.DataFrame(columns = ['time'])
    
    #Convert the dates into a Julian date format
    date = paramsDF['time']
    dateList = []
    for i in date:
        jd = Time(i,format='jd')
        dateList.append(jd)
        
    #Get the Horizons data for the object at the times it was observed
    primary = Horizons(id=objectName,location=None,epochs=dateList)
    
    updatedDF['time'] = paramsDF['time']-primary.vectors()['lighttime']
    
    #Pull all data from csv file
    RA_Prim = np.array(paramsDF['RA-Primary'])
    DEC_Prim = np.array(paramsDF['DEC-Primary'])
    
    deltaRA_Paha = np.array(paramsDF['Delta-RA_Paha'])
    deltaDEC_Paha = np.array(paramsDF['Delta-DEC_Paha'])
    deltaRA_Hiisi = np.array(paramsDF['Delta-RA_Hiisi'])
    deltaDEC_Hiisi = np.array(paramsDF['Delta-DEC_Hiisi'])
    
    RA_Paha_err = np.array(paramsDF['Delta-RA_Paha-err'])
    DEC_Paha_err = np.array(paramsDF['Delta-DEC_Paha-err'])
    RA_Hiisi_err = np.array(paramsDF['Delta-RA_Hiisi-err'])
    DEC_Hiisi_err = np.array(paramsDF['Delta-DEC_Hiisi-err'])

    
    #Convert the deltas back to degrees from arcseconds

    RA_Paha = RA_Prim+deltaRA_Paha/3600/np.cos(DEC_Prim*u.degree)
    DEC_Paha = DEC_Prim + deltaDEC_Paha/3600
    
    RA_Paha_err = RA_Prim+(deltaRA_Paha+RA_Paha_err)/3600/np.cos(DEC_Prim*u.degree)
    DEC_Paha_err = DEC_Prim + (deltaDEC_Paha+DEC_Paha_err)/3600
      
    RA_Hiisi = RA_Prim+deltaRA_Hiisi/3600/np.cos(DEC_Prim*u.degree)
    DEC_Hiisi = DEC_Prim + deltaDEC_Hiisi/3600
    
    RA_Hiisi_err = RA_Prim+(deltaRA_Hiisi+RA_Hiisi_err)/3600/np.cos(DEC_Prim*u.degree)
    DEC_Hiisi_err = DEC_Prim + (deltaDEC_Hiisi+DEC_Hiisi_err)/3600
    
    radf = pd.DataFrame()
    radf['RA-Prim'] = RA_Prim
    radf['DEC-Prim'] = DEC_Prim
    radf['RA-Paha'] = RA_Paha
    radf['DEC-Paha'] = DEC_Paha
    radf['RA-Hiisi'] = RA_Hiisi
    radf['DEC-Hiisi'] = DEC_Hiisi
    
    #print(radf)
    
    #Essentially we define where the object is in our RA/DEC coordinate system. ICRS is the system our coordinates are in.
    dist = primary.vectors()['range']
    PahaC = SkyCoord(ra=RA_Paha*u.degree, dec=DEC_Paha*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    HiisiC = SkyCoord(ra=RA_Hiisi*u.degree, dec=DEC_Hiisi*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    
    PahaC_err = SkyCoord(ra=RA_Paha_err*u.degree, dec=DEC_Paha_err*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    HiisiC_err = SkyCoord(ra=RA_Hiisi_err*u.degree, dec=DEC_Hiisi_err*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    
    primC = SkyCoord(ra=RA_Prim*u.degree, dec=DEC_Prim*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    
    #Transform your icrs frame to a J2000 latitude and longitude ecliptic frame
    PahaEcl = PahaC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    HiisiEcl = HiisiC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    
    PahaEcl_err = PahaC_err.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    HiisiEcl_err = HiisiC_err.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    
    primEcl = primC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    
    Lat_Prim = primEcl.lat.degree
    Long_Prim = primEcl.lon.degree
    
    Lat_Paha = PahaEcl.lat.degree
    Long_Paha = PahaEcl.lon.degree
    
    Lat_Hiisi = HiisiEcl.lat.degree
    Long_Hiisi = HiisiEcl.lon.degree
    
    Lat_Paha_err = PahaEcl_err.lat.degree
    Long_Paha_err = PahaEcl_err.lon.degree
    
    Lat_Hiisi_err = HiisiEcl_err.lat.degree
    Long_Hiisi_err = HiisiEcl_err.lon.degree
    
    DeltaLat_Paha = (Lat_Paha-Lat_Prim)*3600
    DeltaLong_Paha = (Long_Paha-Long_Prim)*np.cos(Lat_Prim*u.degree)*3600
    
    DeltaLat_Hiisi = (Lat_Hiisi-Lat_Prim)*3600
    DeltaLong_Hiisi = (Long_Hiisi-Long_Prim)*np.cos(Lat_Prim*u.degree)*3600
    
    Lat_Paha_err_arc = (Lat_Paha_err-Lat_Paha)*3600
    Long_Paha_err_arc = (Long_Paha_err-Long_Paha)*3600
    
    Lat_Hiisi_err_arc = (Lat_Hiisi_err-Lat_Hiisi)*3600
    Long_Hiisi_err_arc = (Long_Hiisi_err-Long_Hiisi)*3600
    
    
    updatedDF['Lat_Prim'] = Lat_Prim
    updatedDF['Long_Prim'] = Long_Prim
    
    updatedDF['DeltaLat_Paha'] = DeltaLat_Paha
    updatedDF['DeltaLong_Paha'] = DeltaLong_Paha
    updatedDF['DeltaLat_Hiisi'] = DeltaLat_Hiisi
    updatedDF['DeltaLong_Hiisi'] = DeltaLong_Hiisi
    
    updatedDF['DeltaLat_Paha_err'] = Lat_Paha_err_arc
    updatedDF['DeltaLong_Paha_err'] = Lat_Paha_err_arc
    updatedDF['DeltaLat_Hiisi_err'] = Lat_Hiisi_err_arc
    updatedDF['DeltaLong_Hiisi_err'] = Long_Hiisi_err_arc
   
    print(updatedDF)
    
    updatedDF.to_csv('NewLempo_LatLon.csv')
    

In [11]:
paramsDF = pd.read_csv('BenecchiData_NotWrong.csv')

convert_to_primary_centric(paramsDF,'Lempo')

           time  Lat_Prim   Long_Prim  DeltaLat_Paha  DeltaLong_Paha  \
0  2.452252e+06 -8.425337  359.203588       0.348254        0.154825   
1  2.452253e+06 -8.421742  359.203270       0.327880        0.185844   
2  2.452820e+06 -8.416388    6.399658       0.061015        0.322115   
3  2.452830e+06 -8.460839    6.428386      -0.189593        0.166141   
4  2.452846e+06 -8.530841    6.368939       0.035395       -0.156362   
5  2.452875e+06 -8.634287    5.957643      -0.084015        0.306374   
6  2.453152e+06 -8.272809    8.185320       0.191881       -0.073251   
7  2.453595e+06 -8.587731   10.852222      -0.241779       -0.129901   
8  2.453923e+06 -8.398790   13.373498       0.149018        0.342713   

   DeltaLat_Hiisi  DeltaLong_Hiisi  DeltaLat_Paha_err  DeltaLong_Paha_err  \
0        0.043521        -0.007876           0.004157            0.004157   
1        0.009329         0.039463           0.002930            0.002930   
2        0.001736        -0.025173           0.0

In [3]:
#This function makes a scatter plot based on the data given (x,y) data points
def plot_scatter(array,dictionary, words):
    for i in words:
            x, y = array[dictionary[i]]
            plt.scatter(x, y, color = 'red')
            if dictionary[i]==2:
                plt.text(x-0.008,y+0.012,i,fontsize=9)
            else:
                plt.text(x+0.002,y+0.006,i,fontsize=9)
    plt.ylabel('Latitude')
    plt.xlabel('Longitude')
    plt.show()